<a href="https://colab.research.google.com/github/Padmini-RK/Yelp-Business-Data-Analysis/blob/main/TopicModellingV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
! pip install --ignore-installed pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 17.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=647215d30bf9507e853b55164fc6b843b07672839996b88cedd1836c79dcdd18
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [3]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.0
      /_/
                        
Using Scala version 2.12.17, OpenJDK 64-Bit Server VM, 11.0.18
Branch HEAD
Compiled by user xinrong.meng on 2023-04-07T02:18:01Z
Revision 87a5442f7ed96b11051d8a9333476d080054e5a0
Url https://github.com/apache/spark
Type --help for more information.


In [4]:
! pip install --ignore-installed spark-nlp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.7/486.7 kB 12.9 MB/s eta 0:00:00


In [5]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!nltk --version

nltk, version 3.8.1


In [7]:
import sparknlp

spark = sparknlp.start()

In [8]:
from pyspark.sql import functions as F

path_to_review_data = '/content/drive/My Drive/ADBMS/review.json'
review_data = spark.read.json(path_to_review_data)

In [9]:
review_data.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

In [68]:
text_col_name = 'text'
b_col = 'business_id'
top_500 = spark.createDataFrame(review_data.head(20000))
#review_text = review_data.select(text_col_name).filter(F.col(text_col_name).isNotNull())
business_specific_review = top_500.select(text_col_name, 'business_id').filter(F.col(b_col) == "GBTPC53ZrG1ZBY3DT8Mbcw")

#k = top_500.select(b_col).groupby(b_col).count().orderBy('count', ascending=False)
#.show(20)
#print(k.collect()[0][0])
#print(k.collect()[0][1])


In [70]:
business_specific_review.count()

187

In [12]:
#review_text.limit(5).show(truncate=90)

In [71]:
from sparknlp.base import DocumentAssembler

documentAssembler = DocumentAssembler() \
     .setInputCol(text_col_name) \
     .setOutputCol('document')

In [72]:
from sparknlp.annotator import Tokenizer

tokenizer = Tokenizer() \
     .setInputCols(['document']) \
     .setOutputCol('tokenized')
     

In [73]:
from sparknlp.annotator import Normalizer

normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

In [74]:
from sparknlp.annotator import LemmatizerModel

lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemmatized')

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [75]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [76]:
from sparknlp.annotator import StopWordsCleaner

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('unigrams') \
     .setStopWords(eng_stopwords)

In [77]:
from sparknlp.annotator import NGramGenerator

ngrammer = NGramGenerator() \
    .setInputCols(['lemmatized']) \
    .setOutputCol('ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

In [78]:
from sparknlp.annotator import PerceptronModel

pos_tagger = PerceptronModel.pretrained('pos_anc') \
    .setInputCols(['document', 'lemmatized']) \
    .setOutputCol('pos')

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]


In [79]:
from sparknlp.base import Finisher

finisher = Finisher() \
     .setInputCols(['unigrams', 'ngrams', 'pos']) 

In [80]:
from pyspark.ml import Pipeline

pipeline = Pipeline() \
     .setStages([documentAssembler,                  
                 tokenizer,
                 normalizer,                  
                 lemmatizer,                  
                 stopwords_cleaner, 
                 pos_tagger,
                 ngrammer,  
                 finisher])

In [81]:
processed_review = pipeline.fit(business_specific_review).transform(business_specific_review)

In [82]:
processed_review.limit(5).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|         business_id|   finished_unigrams|     finished_ngrams|        finished_pos|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|We have been here...|GBTPC53ZrG1ZBY3DT...|[twice, brunch, e...|[we, have, be, he...|[PRP, VBP, VB, RB...|
|First meal in New...|GBTPC53ZrG1ZBY3DT...|[first, meal, new...|[first, meal, in,...|[JJ, NN, IN, JJ, ...|
|Service is excell...|GBTPC53ZrG1ZBY3DT...|[service, excelle...|[service, be, exc...|[NN, VB, JJ, NN, ...|
|This was our seco...|GBTPC53ZrG1ZBY3DT...|[second, choice, ...|[this, be, we, se...|[DT, VB, PRP, JJ,...|
|Great oyster happ...|GBTPC53ZrG1ZBY3DT...|[great, oyster, h...|[great, oyster, h...|[JJ, NN, JJ, NN, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [83]:
from pyspark.sql import types as T

udf_join_arr = F.udf(lambda x: ' '.join(x), T.StringType())
processed_review  = processed_review.withColumn('finished_pos', udf_join_arr(F.col('finished_pos')))
     

In [84]:
pos_documentAssembler = DocumentAssembler() \
     .setInputCol('finished_pos') \
     .setOutputCol('pos_document')

In [85]:
pos_tokenizer = Tokenizer() \
     .setInputCols(['pos_document']) \
     .setOutputCol('pos')

In [86]:
pos_ngrammer = NGramGenerator() \
    .setInputCols(['pos']) \
    .setOutputCol('pos_ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

In [87]:
pos_finisher = Finisher() \
     .setInputCols(['pos', 'pos_ngrams']) 

In [88]:
pos_pipeline = Pipeline() \
     .setStages([pos_documentAssembler,                  
                 pos_tokenizer,
                 pos_ngrammer,  
                 pos_finisher])
     

In [89]:
processed_review = pos_pipeline.fit(processed_review).transform(processed_review)

In [90]:
processed_review.columns

['text',
 'business_id',
 'finished_unigrams',
 'finished_ngrams',
 'finished_pos',
 'finished_pos_ngrams']

In [91]:
processed_review.select('finished_ngrams', 'finished_pos_ngrams').limit(5).show()

+--------------------+--------------------+
|     finished_ngrams| finished_pos_ngrams|
+--------------------+--------------------+
|[we, have, be, he...|[PRP, VBP, VB, RB...|
|[first, meal, in,...|[JJ, NN, IN, JJ, ...|
|[service, be, exc...|[NN, VB, JJ, NN, ...|
|[this, be, we, se...|[DT, VB, PRP, JJ,...|
|[great, oyster, h...|[JJ, NN, JJ, NN, ...|
+--------------------+--------------------+



In [92]:
def filter_pos(words, pos_tags):
    return [word for word, pos in zip(words, pos_tags) 
            if pos in ['JJ', 'NN', 'NNS', 'VB', 'VBP']]

udf_filter_pos = F.udf(filter_pos, T.ArrayType(T.StringType()))

In [93]:
processed_review = processed_review.withColumn('filtered_unigrams',
                                               udf_filter_pos(F.col('finished_unigrams'), 
                                                              F.col('finished_pos')))
     

In [94]:
processed_review.select('filtered_unigrams').limit(5).show(truncate=90)

+------------------------------------------------------------------------------------------+
|                                                                         filtered_unigrams|
+------------------------------------------------------------------------------------------+
|           [brunch, enjoy, delicious, grit, scallions, room, freeze, sure, bring, sweater]|
|[first, meal, orleans, lunch, seafood, oyster, meat, claw, shrimp, come, surprisingly, ...|
|[service, excellent, atmosphere, great, raw, bar, good, alaskan, crab, leg, well, splur...|
|[choice, rio, mar, mardi, week, hell, time, side, parade, stay, great, service, overbea...|
|[great, oyster, happy, hour, pm, dozen, glass, tip, value, hear, cup, totally, staff, n...|
+------------------------------------------------------------------------------------------+



In [95]:
def filter_pos_combs(words, pos_tags):
    return [word for word, pos in zip(words, pos_tags) 
            if (len(pos.split('_')) == 2 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS']) \
            or (len(pos.split('_')) == 3 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                  pos.split('_')[2] in ['NN', 'NNS'])]
    
udf_filter_pos_combs = F.udf(filter_pos_combs, T.ArrayType(T.StringType()))
     

In [96]:
processed_review = processed_review.withColumn('filtered_ngrams',
                                               udf_filter_pos_combs(F.col('finished_ngrams'),
                                                                    F.col('finished_pos_ngrams')))
     

In [97]:
processed_review.select('filtered_ngrams').limit(5).show(truncate=90)

+------------------------------------------------------------------------------------------+
|                                                                           filtered_ngrams|
+------------------------------------------------------------------------------------------+
|                                             [have_try, be_delicious, sour_cream, be_sure]|
|[first_meal, new_orleans, lunch_special, seafood_stew, oyster_meat, meat_crab, crab_cla...|
|[be_excellent, excellent_atmosphere, be_great, raw_bar, be_fantastic, good_alaskan, ala...|
|[second_choice, rio_mar, mardi_gra, gra_week, other_side, stay_nonetheless, great_meal,...|
|[great_oyster, oyster_happy, happy_hour, dozen_oyster, tip_great, great_value, good_tog...|
+------------------------------------------------------------------------------------------+



In [98]:
from pyspark.sql.functions import concat

processed_review = processed_review.withColumn('final', 
                                               concat(F.col('filtered_unigrams'), 
                                                      F.col('filtered_ngrams')))
     

In [99]:
processed_review.select('final').limit(5).show(truncate=90)

+------------------------------------------------------------------------------------------+
|                                                                                     final|
+------------------------------------------------------------------------------------------+
|[brunch, enjoy, delicious, grit, scallions, room, freeze, sure, bring, sweater, have_tr...|
|[first, meal, orleans, lunch, seafood, oyster, meat, claw, shrimp, come, surprisingly, ...|
|[service, excellent, atmosphere, great, raw, bar, good, alaskan, crab, leg, well, splur...|
|[choice, rio, mar, mardi, week, hell, time, side, parade, stay, great, service, overbea...|
|[great, oyster, happy, hour, pm, dozen, glass, tip, value, hear, cup, totally, staff, n...|
+------------------------------------------------------------------------------------------+



In [100]:
from pyspark.ml.feature import CountVectorizer

tfizer = CountVectorizer(inputCol='final', outputCol='tf_features')
tf_model = tfizer.fit(processed_review)
tf_result = tf_model.transform(processed_review)

In [101]:
from pyspark.ml.feature import IDF

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

In [102]:
from pyspark.ml.clustering import LDA

num_topics = 6
max_iter = 10

lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(tfidf_result)
     

In [103]:
vocab = tf_model.vocabulary

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))
     

In [104]:
num_top_words = 10

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=90)

+-----+------------------------------------------------------------------------------------------+
|topic|                                                                                topicWords|
+-----+------------------------------------------------------------------------------------------+
|    0|      [even, meat, solid, make, great_service, casual, new_orleans, fare, orleans, always]|
|    1|  [oyster, fry_oyster, fry, good, oyster_salad, take, salad, get, enjoy, fry_oyster_salad]|
|    2|                   [however, lüke, come, second, night, fave, quickly, see, be_worth, bar]|
|    3|        [shrimp, wild_mushroom, thin, waiter, wild, hard, tender, mushroom, raw_bar, pork]|
|    4|                 [order, try, burger, be_great, beer, crab, be_perfect, great, beet, also]|
|    5|[coffee, happy, amazing, customer_service, turn, adequate, meatlovers, drink_make, besh...|
+-----+------------------------------------------------------------------------------------------+



In [47]:
topicdf = topics.select('topicWords')
topicdf.collect()[0][0]

['holiday_special',
 'unable',
 'imaginethe_service',
 'couple_friend',
 'thing',
 'thinly',
 'single_color',
 'appt_im',
 'okay',
 'build_vietnamese']